In [34]:
import importlib
import core_images
import os

importlib.reload(core_images)

#clear directories
!rm data/slices/* data/cores/*

#get list of all images in directories
image_dirs = ["data/Core Images/3-03- 11/", "data/Core Images/42-29- 6/"]#,"data/Core Images/21-20a- 1/",","data/Core Images/206-01- 2/"]
images = []
for dir in image_dirs:
    files = os.listdir(dir)
    dir_images = [os.path.join(dir, f) for f in files]
    images += dir_images

images = sorted(images)

# images = ["data/Core Images/3-03- 11/S00066842.jpg"]

core_x = 730
core_x_size = 6240
core_x_cm = 955
core_y_size = 865
core_x = (core_x,core_x + core_x_size)
core_y= (
    (1470,1470 + core_y_size),
    (2550,2550 + core_y_size),
    (3615,3615 + core_y_size)
)

slice = 224
step = slice/4

cores = core_images.CoreImages(images, core_x, core_y, core_x_cm, slice, step)

labels = "data/core_labels.csv"

cores.slice_cores("data/cores/", "data/slices/", labels, verbose = 1)

Core labels:
      photo_ID  n_core  length  type Unnamed: 4
0    S00066842       1       0     3        NaN
1    S00066842       1     130     1        NaN
2    S00066842       1     195     2        NaN
3    S00066842       1     315     1        NaN
4    S00066842       1     465     3        NaN
..         ...     ...     ...   ...        ...
208  S00085353       2     358     1        NaN
209  S00085353       3       0     1        NaN
210  S00085353       3     145     2        NaN
211  S00085353       3     190    10        NaN
212  S00085353       3     300     1        NaN

[213 rows x 5 columns]
Processing core photo: data/Core Images/3-03- 11/S00066841.jpg
Processing core photo: data/Core Images/3-03- 11/S00066842.jpg
Processing core photo: data/Core Images/3-03- 11/S00066843.jpg
Processing core photo: data/Core Images/3-03- 11/S00066844.jpg
Processing core photo: data/Core Images/3-03- 11/S00066845.jpg
Processing core photo: data/Core Images/3-03- 11/S00066846.jpg
Processin

In [56]:
import pandas as pd
df_metadata = pd.read_csv("data/slice_metadata.csv")
photo_ID = "S00085351"
n_core = 2
n_slice = 65

In [57]:
df_metadata.loc[(df_metadata["n_core"] == n_core) & (df_metadata["photo_ID"] == photo_ID) & (df_metadata["n_slice"] == n_slice)]["label"]

5141    9
Name: label, dtype: object

In [58]:
label = int(df_metadata.loc[(df_metadata["n_core"] == n_core) & (df_metadata["photo_ID"] == photo_ID) & (df_metadata["n_slice"] == n_slice)]["label"])

9